In [1]:
#!pip install alpha_vantage

In [1]:
from alpha_vantage.timeseries import TimeSeries
import pandas as pd
import numpy as np
import warnings
import time
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_excel('Market Data.xlsx')
df = df.drop(['Unnamed: 0'], axis = 1)
df = df[(df['Iid'] == '01')]  # | (df['Iid'] == '02')
df = df.sort_values(['GVKey', 'DataDate'])
df.reset_index(inplace = True)

In [58]:
two = np.zeros((65,2))
one = np.array(['a' for _ in range(65)], dtype=object).reshape(65,1)
table = np.hstack((one,two))

In [3]:
ts = TimeSeries(key='UKF4TSUWJC3TIMY9', output_format='pandas')

symbols = pd.read_excel('symbols.xlsx')
symbols = np.array(symbols['Symbol'])

s = 0
for symbol in symbols[s:len(symbols)]:    
    data, _ = ts.get_monthly_adjusted(symbol = symbol)
    new = data.loc['2017-08-31':'2017-01-01']
    new = new.sort_values('date')
    new['trt1m'] = new['5. adjusted close'].pct_change()
    new = np.array(new['trt1m'].dropna())

    Assets_table = df.pivot_table(
                        index='DataDate', 
                        columns='GVKey', 
                        values='trt1m').applymap(float)

    Assets_table = Assets_table.loc['2017-02-28':'2017-08-31', :]
    Assets_table = Assets_table.dropna(axis = 1)
    Assets_table = Assets_table.div(100, axis=0)
    Assets_table = Assets_table.sub(new, axis=0)
    Assets_table = Assets_table.apply(lambda x: x**2)

    Assets_table.loc['itog', :] = Assets_table.sum()

    indices = np.where(Assets_table==Assets_table.loc['itog', :].min())
    sec_gvkey = Assets_table.columns[indices[1][0]]
    sec_error = Assets_table.loc['itog', :].min()

    table[s][0] = symbol
    table[s][1] = sec_gvkey
    table[s][2] = sec_error
    s += 1
    time.sleep(12)
    

In [5]:
deanon = pd.read_excel('my_excel_file.xlsx')
deanon.head(10)

,Company,Label,GVKey,Error
0,Berkshire Hathaway Inc,BRK,2176,0.000406
1,DXC Technology Co.,DXC,3336,0.000018
2,Aon plc,AON,3221,0.000009
3,Pentair plc,PNR,8463,0.000006
4,CenturyLink Inc.,CTL,2884,0.000005
5,PPL Corporation,PPL,8455,0.000003
6,Progressive Corporation,PGR,13341,0.000002
7,Franklin Resources Inc.,BEN,4885,0.000002
8,Helmerich & Payne Inc.,HP,5581,0.000002
9,L Brands Inc.,LB,6733,0.000002
